## Problem 4: Part A

You arrive at the airport only to realize that you grabbed your North Pole Credentials instead of your passport. While these documents are extremely similar, North Pole Credentials aren't issued by a country and therefore aren't actually valid documentation for travel in most of the world.

It seems like you're not the only one having problems, though; a very long line has formed for the automatic passport scanners, and the delay could upset your travel itinerary.

Due to some questionable network security, you realize you might be able to solve both of these problems at the same time.

The automatic passport scanners are slow because they're having trouble detecting which passports have all required fields. The expected fields are as follows:

byr (Birth Year) <br />
iyr (Issue Year)<br />
eyr (Expiration Year)<br />
hgt (Height)<br />
hcl (Hair Color)<br />
ecl (Eye Color)<br />
pid (Passport ID)<br />
cid (Country ID)<br />

Passport data is validated in batch files (your puzzle input). Each passport is represented as a sequence of key:value pairs separated by spaces or newlines. Passports are separated by blank lines.

Here is an example batch file containing four passports:

ecl:gry pid:860033327 eyr:2020 hcl:#fffffd
byr:1937 iyr:2017 cid:147 hgt:183cm

iyr:2013 ecl:amb cid:350 eyr:2023 pid:028048884
hcl:#cfa07d byr:1929

hcl:#ae17e1 iyr:2013
eyr:2024
ecl:brn pid:760753108 byr:1931
hgt:179cm

hcl:#cfa07d eyr:2025 pid:166559648
iyr:2011 ecl:brn hgt:59in <br />

The first passport is valid - all eight fields are present. The second passport is invalid - it is missing hgt (the Height field).

The third passport is interesting; the only missing field is cid, so it looks like data from North Pole Credentials, not a passport at all! Surely, nobody would mind if you made the system temporarily ignore missing cid fields. Treat this "passport" as valid.

The fourth passport is missing two fields, cid and byr. Missing cid is fine, but missing any other field is not, so this passport is invalid.

According to the above rules, your improved system would report 2 valid passports.

Count the number of valid passports - those that have all required fields. Treat cid as optional. In your batch file, how many passports are valid?

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = open('data/problem_4_values.txt', 'r')
data = data.read()
data = data.split('\n\n')
data = [i.replace('\n', ' ') for i in data]

In [3]:
required_fields = ['iyr', 'pid', 'eyr', 'hcl', 'ecl', 'byr', 'hgt']

In [4]:
has_fields = []

for passport in data:
    if all((field in passport) for field in required_fields):
        has_fields.append(passport)

In [5]:
len(has_fields)

190

In [6]:
print ('THE CORRECT ANSWER IS', len(has_fields), "!!!")

THE CORRECT ANSWER IS 190 !!!


## Problem 4: Part B

The line is moving more quickly now, but you overhear airport security talking about how passports with invalid data are getting through. Better add some data validation, quick!

You can continue to ignore the cid field, but each other field has strict rules about what values are valid for automatic validation:

byr (Birth Year) - four digits; at least 1920 and at most 2002. <br />
iyr (Issue Year) - four digits; at least 2010 and at most 2020.<br />
eyr (Expiration Year) - four digits; at least 2020 and at most 2030.<br />
hgt (Height) - a number followed by either cm or in:<br />
If cm, the number must be at least 150 and at most 193.<br />
If in, the number must be at least 59 and at most 76.<br />
hcl (Hair Color) - a # followed by exactly six characters 0-9 or a-f.<br />
ecl (Eye Color) - exactly one of: amb blu brn gry grn hzl oth.<br />
pid (Passport ID) - a nine-digit number, including leading zeroes.<br />
cid (Country ID) - ignored, missing or not.<br />

Your job is to count the passports where all required fields are both present and valid according to the above rules. Here are some example values:

![](data/images/problem_4_a.jpg) <br/>

Here are some invalid passports:

![](data/images/problem_4_b.jpg) <br />

Here are some valid passports:

![](data/images/problem_4_c.jpg) <br />

Count the number of valid passports - those that have all required fields and valid values. Continue to treat cid as optional. In your batch file, how many passports are valid?

### Rules summary

byr (Birth Year) - four digits; at least 1920 and at most 2002. <br />
iyr (Issue Year) - four digits; at least 2010 and at most 2020.<br />
eyr (Expiration Year) - four digits; at least 2020 and at most 2030.<br />
hgt (Height) - a number followed by either cm or in:<br />
If cm, the number must be at least 150 and at most 193.<br />
If in, the number must be at least 59 and at most 76.<br />
hcl (Hair Color) - a # followed by exactly six characters 0-9 or a-f.<br />
ecl (Eye Color) - exactly one of: amb blu brn gry grn hzl oth.<br />
pid (Passport ID) - a nine-digit number, including leading zeroes.<br />
cid (Country ID) - ignored, missing or not.<br />

In [7]:
data2 = []

for passport in has_fields:
    fields = passport.split(' ')
    fields = [field.replace(':', ' ').split(' ') for field in fields]
    fields = dict(fields)
    
    fields['byr'] = int(fields['byr'])
    fields['iyr'] = int(fields['iyr'])
    fields['eyr'] = int(fields['eyr'])
    
    data2.append(dict(fields))

In [8]:
def byr_test(byr):
    return (
        (len(str(byr)) == 4) & # unnecessary because the 2nd condition guarantees only values with 4 digits will pass
        (1920 <= byr <= 2002)
    )

In [9]:
def iyr_test(iyr):
    return (
        (len(str(iyr)) == 4) &
        (2010 <= iyr <= 2020)
    )

In [10]:
def eyr_test(eyr):
    return (
        (len(str(eyr)) == 4) &
        (2020 <= eyr <= 2030)
    )

In [11]:
def hgt_test(hgt):
    find_cm = hgt.find('cm')
    find_in = hgt.find('in')
    
    if (find_cm == -1) & (find_in == -1):
        return False
    elif find_cm != -1:
        try:
            return (150 <= int(hgt[:find_cm]) <= 193)
        except:
            return False
    else:
        try:
            return (59 <= int(hgt[:find_in]) <= 76)
        except:
            return False

In [12]:
def hcl_test(hcl):
    valid_chars = 'abcdef0123456789'
    
    if (hcl[0] == '#') & (len(hcl[1:]) == 6) & all((char in valid_chars) for char in hcl[1:]):
        return True
    else:
        return False

In [13]:
def ecl_test(ecl):
    valids = list('amb blu brn gry grn hzl oth'.split(' '))
    
    for valid in valids:
        if ecl == valid:
            return True
    return False

In [14]:
def pid_test(pid):
    
    try:
        int(pid)
    except:
        return False
    else:
        if len(str(pid)) == 9:
            return True
    return False

In [15]:
super_valids = []

for passport in data2:
    byr = byr_test(passport['byr'])
    iyr = iyr_test(passport['iyr'])
    eyr = eyr_test(passport['eyr'])
    hgt = hgt_test(passport['hgt'])
    hcl = hcl_test(passport['hcl'])
    ecl = ecl_test(passport['ecl'])
    pid = pid_test(passport['pid'])
    
    if byr & iyr & eyr & hgt & hcl & ecl & pid:
        super_valids.append(passport)

In [16]:
len(super_valids)

121

In [17]:
print ("THE CORRECT ANSWER IS", len(super_valids), "!!!")

THE CORRECT ANSWER IS 121 !!!
